In [2]:
import numpy as np
import pandas as pd
import math
test=pd.read_csv("/content/test.csv")
train=pd.read_csv("/content/train.csv")

In [18]:
print(test.shape)
test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
df=pd.DataFrame(train)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [26]:
print(df.isnull().sum().sort_values(ascending=False))

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64


In [3]:
def cleanDatasets(data,isTest):
  data=data.drop(['Name','PassengerId','Ticket','Fare','Cabin'],axis=1)

  for i in range(0,data.shape[0]):
    if data['Sex'][i] =="female":
      data['Sex'][i]=1

    else:
      data['Sex'][i]=0

    if data['Embarked'][i] =='S':
      data['Embarked'][i]=0

    elif data['Embarked'][i] =='C':
      data['Embarked'][i]=1

    else:
      data['Embarked'][i]=2 

    if math.isnan(data['Age'][i]):
      data['Age'][i]=data['Age'].mean()

    if math.isnan(data['Pclass'][i]):
      data['Pclass'][i]=data['Pclass'].mean()

    if math.isnan(data['SibSp'][i]):
      data['SibSp'][i]=data['SibSp'].mean()



  data['Embarked'][0] = 0
  x_data=data.values
  x_data=np.float32(np.transpose(x_data))

  if isTest:
    return x_data

  p_data=np.array([data['Survived'].values])
  p_data=np.float32(np.transpose(p_data))
  return x_data,p_data

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
x_train_data, p_train_data=cleanDatasets(train, False)
x_test_data=cleanDatasets(test,True)

In [8]:
x_train_data=np.delete(x_train_data,0,0)

In [9]:
x_train_data=np.transpose(x_train_data)
x_test_data=np.transpose(x_test_data)

In [10]:
p_train_data=np.reshape(p_train_data, (len(p_train_data) ))

In [11]:
import sklearn
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler(feature_range=(-1,1)).fit(np.concatenate((x_train_data,x_test_data),axis=0))

x_train_data=scaler.transform(x_train_data)
x_test_data=scaler.transform(x_test_data)

In [13]:
#!pip install catboost
import catboost
from catboost import CatBoostClassifier

model=CatBoostClassifier(iterations=2000,learning_rate=0.5,eval_metric='Accuracy',loss_function='Logloss',max_depth=3)
model.fit(x_train_data[0:712],p_train_data[0:712], eval_set=(x_train_data[712:891], p_train_data[712:891]), use_best_model=False)


     |████████████████████████████████| 76.1 MB 64 kB/s 
0:	learn: 0.8075843	test: 0.8268156	best: 0.8268156 (0)	total: 47.5ms	remaining: 1m 35s
1:	learn: 0.8174157	test: 0.8379888	best: 0.8379888 (1)	total: 48.1ms	remaining: 48.1s
2:	learn: 0.8230337	test: 0.8603352	best: 0.8603352 (2)	total: 48.7ms	remaining: 32.4s
3:	learn: 0.8230337	test: 0.8603352	best: 0.8603352 (2)	total: 51.5ms	remaining: 25.7s
4:	learn: 0.8244382	test: 0.8547486	best: 0.8603352 (2)	total: 53.2ms	remaining: 21.2s
5:	learn: 0.8258427	test: 0.8603352	best: 0.8603352 (2)	total: 54.7ms	remaining: 18.2s
6:	learn: 0.8314607	test: 0.8603352	best: 0.8603352 (2)	total: 56.2ms	remaining: 16s
7:	learn: 0.8258427	test: 0.8603352	best: 0.8603352 (2)	total: 57.7ms	remaining: 14.4s
8:	learn: 0.8258427	test: 0.8603352	best: 0.8603352 (2)	total: 59.2ms	remaining: 13.1s
9:	learn: 0.8356742	test: 0.8770950	best: 0.8770950 (9)	total: 59.7ms	remaining: 11.9s
10:	learn: 0.8272472	test: 0.8826816	best: 0.8826816 (10)	total: 60.1ms	re

In [15]:
predictions=model.predict(x_test_data)
id=test['PassengerId'].values
submission=pd.DataFrame(np.transpose([id,predictions]))
submission.columns=['PassengerId','Survived']
submission['Survived']=np.int32(np.round(submission['Survived']))
submission['PassengerId']=np.int32(np.round(submission['PassengerId']))

In [16]:
Sonucs=submission
print(Sonucs)

     PassengerId  Survived
0            892         0
1            893         0
2            894         1
3            895         1
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


In [ ]:
# Sonucs.to_csv('deneme.csv',index=False) # save to file